# Creates and simulates a circuit equivalent to a Bell inequality test

In [1]:
# Imports
import numpy as np
import cirq

In [7]:
def main():
    # Create circuit
    circuit = make_bell_test_circuit()
    print('Circuit:')
    print(circuit)

    # Run simulations
    print()
    repetitions = 1000
    print('Simulating {} repetitions ...'.format(repetitions))
    result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

    # Collect results
    a = np.array(result.measurements['a'][:,0])
    b = np.array(result.measurements['b'][:,0])
    x = np.array(result.measurements['x'][:,0])
    y = np.array(result.measurements['y'][:,0])

    # Compute the winning percentage
    outcomes = a ^ b == x & y
    win_percent = len([e for e in outcomes if e]) * 100 / repetitions

    # Print data
    print()
    print('Results')
    print('a:', bitstring(a))
    print('b:', bitstring(b))
    print('x:', bitstring(x))
    print('y:', bitstring(y))
    print('(a XOR b) == (x AND y):\n', bitstring(outcomes))
    print('Win rate: {}%'.format(win_percent))

In [5]:
def make_bell_test_circuit():
    # Qubits for Alice, Bob, and referees
    alice = cirq.GridQubit(0,0)
    bob = cirq.GridQubit(1,0)
    alice_referee = cirq.GridQubit(0,1)
    bob_referee = cirq.GridQubit(1,1)

    circuit = cirq.Circuit()

    # Prepare shared entangled state between Alice and Bob
    circuit.append([
        cirq.H(alice),
        cirq.CNOT(alice,bob),
        cirq.X(alice) ** -0.25,
    ])

    # Referee flip coins
    circuit.append([
        cirq.H(alice_referee),
        cirq.H(bob_referee),
    ])

    # Players do a sqrt(X) based on their referee's coin
    circuit.append([
        cirq.CNOT(alice_referee, alice)**0.5,
        cirq.CNOT(bob_referee,bob)**0.5,
    ])

    # Then results are recorded
    circuit.append([
        cirq.measure(alice, key='a'),
        cirq.measure(bob, key='b'),
        cirq.measure(alice_referee, key='x'),
        cirq.measure(bob_referee, key='y'),
    ])

    return circuit

In [6]:
def bitstring(bits):
    return ''.join('1' if e else '0' for e in bits)

In [8]:
if __name__ == '__main__':
    main()

Circuit:
(0, 0): ───H───@───X^-0.25───X^0.5────M('a')───
               │             │
(0, 1): ───H───┼─────────────@────────M('x')───
               │
(1, 0): ───────X───X^0.5─────M('b')────────────
                   │
(1, 1): ───H───────@─────────M('y')────────────

Simulating 1000 repetitions ...

Results
a: 0110000011000010001011001010110000001000011111011010001110100000001010011001000011101000001111001101000000011000010000111001000000100110110111010110011100011111100001010010001100000101101000110011001110101101100010001100111010111001010101011001101111110001010011101000010010000110111100110100101100000101110000000000110010000001101110100001100100001010001110111100101000011100001110001010010010100111011011100100010011011000101100100100010111110110111011101000100001010100000000111000000010010010100101011111111000010010110101001100110011011011010000000000000010111001010010001010011100000100101110110101101010111001110000110101000100111010000100100011110111011101010111100000000010111

The output of this program is the following. The bitstrings for Alice and Bob are shown in the first two lines (a and b). These are the bits each palyer sends back to the referee (the underscore indicates the 0 bit). The next two lines (x and y) show the bitstrings sent to Alice and Bob by the referee. Finally, the winning condition a(x) $\oplus$ b(y) = xy is shown as a bitstring for each round. The win rate is computed from the number of 1s in this bitstring, which indicate a win.